In [8]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [22]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [14]:
len(text)

556949

In [15]:
vocabulary = set(text)
len(vocabulary)

92

In [16]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [18]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [23]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, encoder_hidden_units = 100, decoder_hidden_units = 100)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3)

Net parameters: 207,195



Epoch:    1, Loss: 2.5410
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' ee  ee  e', ' e  ee  e ', '   ee  e e', '  ee  e ee', ' te  e ee ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
tensor([ -9.7900, -10.7400, -11.6600,  -8.9900,  -8.7400], device='cuda:0')
sample
['   os. o .', '\nmk-tKeepa', 'hincfednue', ' nhe ls ot', '. 2P Ka\n\nl']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['         t', '         t', '         t', '         t', '         t'],
 ['        t ', '        t ', '        t ', '        t ', '        t '],
 ['         a', '         o', '         o', '       t  ', '         .'],
 ['         o', '         a', '       t  ', '         a', '         a']]
tensor([[ -9.7900, -10.7400, -11.6600,  -8.9900,  -8.7400],
        [-12.8400, -13.7600, -14.5000, -12.3300, -11.9000],
        [-13.2300, -14.0500, -14.7300, -12.5800, -12.3000],
        [-13.3600, -14


Epoch:    2, Loss: 1.9396
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' h aose te', 'ehaose te ', ' tote te e', '  te te ee', ' te te eet']
greedy_search
[' the the t', 'e the the ', ' the the t', ' the the t', ' the the t']
tensor([-12.1800, -11.9400, -11.3000, -11.5500, -11.6200], device='cuda:0')
sample
['xly Qiunce', 'oteripnljl', '\n\nen me in', '.g  inlewi', 'thilg bay;']
beam_search
[['s the the ', 'e the the ', 'e the the ', 'e the the ', ' the the t'],
 [' the the t', 'the the th', 'the the th', ' the the t', 'e the the '],
 ['e the the ', 'e the thes', ' the the t', 'the the th', 'the the th'],
 [' the thes ', 'e the ther', ' the thes ', ' the thes ', ' the thes '],
 [' the the p', 'the the to', ' the the a', ' the the a', ' the the a']]
tensor([[-11.8200, -11.9400, -10.6400, -11.2400, -11.6200],
        [-12.1800, -12.0400, -11.2000, -11.5500, -11.6800],
        [-12.2000, -13.1600, -11.3000, -11.5700, -11.7900],
        [-12.2600, -13.


Epoch:    3, Loss: 1.6564
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aaserte', 'ehauserte ', ' aalerte e', ' alerte ea', ' lerte e.t']
greedy_search
[' the the t', 'enthe the ', ' and the K', ' andere th', ' the the t']
tensor([-12.6800, -12.8900, -13.5100, -15.6400, -13.0900], device='cuda:0')
sample
['es.\nK Q4\na', 'bch PxBpin', 'e coves, d', 'Qt4. 1. R.', 'ok\n\nm\n fxa']
beam_search
[[' the Black', 'the the th', 'e the the ', 'he the the', ' the there'],
 ['s the the ', 'the thite ', 'he the the', 'and the th', 'the the th'],
 [' the there', 'and the th', 'ing the Kt', 'his the th', 'the there '],
 ['s the ther', 'the the pa', 'ing the th', 'and the Kt', ' the the t'],
 [' the the t', 'the there ', 'ing the Bl', 'he the thi', 'the theres']]
tensor([[-11.0700, -11.8600, -10.8600, -11.8000, -12.2500],
        [-12.1100, -12.1200, -11.1700, -12.4600, -12.6900],
        [-12.2600, -12.1700, -11.5100, -12.5300, -12.9100],
        [-12.6500, -


Epoch:    4, Loss: 1.5154
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' k autenty', 'thoutenty ', ' autente  ', 'husente   ', ' lente e.t']
greedy_search
[' the pawn ', 'the the pa', ' and the K', 'he pawn th', ' the Kting']
tensor([-13.3200, -12.5600, -12.3500, -13.4100, -13.4700], device='cuda:0')
sample
["n. 's QBac", 'veint amve', 'ack hin pr', 'as dede bi', ' timeind w']
beam_search
[[' the Black', 'n the Blac', 'ing the Kt', 'the Black ', 'the Black '],
 ['s the Blac', 'n the the ', 'ing the Bl', "the Black'", "the Black'"],
 ['s the King', 'n the ther', 'ing the th', 'the Black,', 'the Black,'],
 ['s the Blak', 'the the th', 'ing the Ki', 'the Black.', 'there the '],
 ['s the Blay', 'the the pa', 'ing the pa', 'the Black\n', 'the Black.']]
tensor([[-10.2700, -10.7600, -10.9900,  -9.0300, -10.3300],
        [-11.0800, -10.9100, -11.2000, -10.8200, -11.8400],
        [-11.9600, -12.2600, -11.5600, -11.0700, -12.0300],
        [-12.7700, -12.3


Epoch:    5, Loss: 1.4288
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['nk autenty', 't tutente ', ' tutente  ', 'tutente   ', 'tsentere.t']
greedy_search
['n the pawn', 't the Kt-K', ' the Kt-Kt', 'the pawn t', 'the the pa']
tensor([-12.1500, -12.9100, -12.6700, -11.7800, -13.1900], device='cuda:0')
sample
['s, ot pawn', 'agnon,\nBla', 'B-B3; 2. K', 'therewibul', 'eng. prest']
beam_search
[['ng.\n\n     ', 'n the Blac', ' the Black', 'the Black ', 'the Black '],
 ['s the Blac', 'n the King', ' the King ', "the Black'", "the Black'"],
 ['s, and the', 'ing the Bl', ' the King.', 'the King. ', 'and the Kt'],
 ['s the pawn', 'ing the Kt', 'ing the Bl', 'the Black\n', 'and the Bl'],
 ['s, and to ', 'ing the pa', 'ing the Kt', 'the Black,', 'the Black\n']]
tensor([[ -8.6700,  -9.6500,  -8.5100,  -7.9200, -10.1700],
        [-10.4400, -10.2700, -10.1300,  -9.3600, -11.2100],
        [-10.5000, -11.2500, -10.6500, -10.1100, -11.7300],
        [-11.8800, -

In [32]:
net.cpu().beam_search(source_1[:10000].cpu(), verbose = 1, batch_size = 50)

(tensor([[[ 1, 80, 73,  ..., 86, 74, 71],
          [ 1, 80, 73,  ..., 80, 86,  5],
          [ 1, 80, 73,  ..., 71, 80, 86],
          [ 1, 80, 73,  ..., 71, 80, 75],
          [ 1, 80, 73,  ..., 71, 85,  5]],
 
         [[ 1, 80,  5,  ..., 69, 77,  5],
          [ 1, 80,  5,  ..., 69, 77, 12],
          [ 1, 80,  5,  ..., 78, 67, 69],
          [ 1, 80,  5,  ..., 69, 77, 17],
          [ 1, 80,  5,  ..., 69, 77,  4]],
 
         [[ 1,  5, 86,  ..., 77, 12, 85],
          [ 1,  5, 86,  ..., 67, 69, 77],
          [ 1,  5, 86,  ..., 77, 17,  5],
          [ 1,  5, 86,  ..., 77,  5, 67],
          [ 1,  5, 86,  ..., 77,  5, 86]],
 
         ...,
 
         [[ 1, 81, 72,  ..., 71,  5, 82],
          [ 1, 81, 72,  ..., 74, 71,  5],
          [ 1, 81, 72,  ..., 71,  5, 79],
          [ 1, 81, 72,  ..., 71,  5, 67],
          [ 1, 81, 72,  ..., 71,  5, 48]],
 
         [[ 1, 81, 72,  ..., 74, 71,  5],
          [ 1, 81, 72,  ..., 74, 71,  5],
          [ 1, 81, 72,  ..., 71,  5, 82],
      